<a href="https://colab.research.google.com/github/jittinabraham/ADSB_signalAnalysis_from-Airplane_using_RTL-SDR/blob/master/TextModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!wget https://github.com/tesseract-ocr/tessdata/raw/main/deu.traineddata
!apt-get install tesseract-ocr-deu

Selecting previously unselected package tesseract-ocr-deu.
(Reading database ... 121796 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-deu_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-deu (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-deu (1:4.00~git30-7274cfa-1.1) ...


In [2]:
!wget https://zenodo.org/record/7869954/files/products_leaflets_512.zip

--2024-02-15 15:04:24--  https://zenodo.org/record/7869954/files/products_leaflets_512.zip
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7869954/files/products_leaflets_512.zip [following]
--2024-02-15 15:04:24--  https://zenodo.org/records/7869954/files/products_leaflets_512.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 3367680859 (3.1G) [application/octet-stream]
Saving to: ‘products_leaflets_512.zip’

products_leaflets_5 100%[===================>]   3.14G  22.6MB/s    in 2m 29s  

2024-02-15 15:06:54 (21.6 MB/s) - ‘products_leaflets_512.zip’ saved [3367680859/3367680859]



In [3]:
!unzip products_leaflets_512.zip

Streaming output truncated to the last 5000 lines.
  inflating: products_leaflets_512/train/738/39037.jpg  
  inflating: products_leaflets_512/train/738/39038.jpg  
  inflating: products_leaflets_512/train/738/39040.jpg  
  inflating: products_leaflets_512/train/738/39041.jpg  
  inflating: products_leaflets_512/train/738/39042.jpg  
  inflating: products_leaflets_512/train/738/39043.jpg  
  inflating: products_leaflets_512/train/738/39050.jpg  
  inflating: products_leaflets_512/train/738/39051.jpg  
  inflating: products_leaflets_512/train/738/39052.jpg  
  inflating: products_leaflets_512/train/738/39053.jpg  
  inflating: products_leaflets_512/train/738/39054.jpg  
  inflating: products_leaflets_512/train/738/39055.jpg  
  inflating: products_leaflets_512/train/738/39056.jpg  
  inflating: products_leaflets_512/train/738/39057.jpg  
  inflating: products_leaflets_512/train/738/39058.jpg  
  inflating: products_leaflets_512/train/738/39059.jpg  
  inflating: products_leaflets_512/tr

In [4]:
import re
import os
import openpyxl
from PIL import Image
import pytesseract
import cv2
import shutil
import tqdm
from tqdm import tqdm
import pandas as pd

In [ ]:
temp_dir = '/usr/share/tesseract-ocr/4.00/tessdata'

# Create the temporary directory
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Set the TESSDATA_PREFIX environment variable
os.environ['TESSDATA_PREFIX'] = temp_dir

In [ ]:

#tesseract configrations in different PSM #3 6 11 12 for RGB images
def textextraction_psm6_RGB(image_file):

  custom_config ='''
  --oem 1
  --psm 6
  -l deu
  -c tessedit_char_blacklist=!@#$^""»«.&*->><<(^){}[]|\\
  '''
  text_extracted=pytesseract.image_to_string(image_file,lang='deu',config=custom_config)
  return text_extracted




In [ ]:
def textextraction_psm12_RGB(image_file):
    os.environ['TESSDATA_PREFIX'] = temp_dir
    custom_config ='''
    --oem 1
    --psm 12
    -l deu
    -c tessedit_char_blacklist=!@#$^""»«.&*->><<(^){}[]|\\
    '''
    text_extracted=pytesseract.image_to_string(image_file,lang='deu',config=custom_config)
    return text_extracted

In [ ]:

#tesseract configrations in different PSM #3 6 11 12 for Gray images
def textextraction_psm6_Gray(image_file):
  custom_config ='''
  --oem 1
  --psm 6
  -l deu
  -c tessedit_char_blacklist=!@#$^""»«.&*->><<(^){}[]|\\
  '''
  image = cv2.imread(image_file)
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  text_extracted=pytesseract.image_to_string(gray_image,lang='deu',config=custom_config)
  return text_extracted




In [ ]:
def textextraction_psm12_Gray(image_file):
    os.environ['TESSDATA_PREFIX'] = temp_dir
    custom_config ='''
    --oem 1
    --psm 12
    -l deu
    -c tessedit_char_blacklist=!@#$^""»«.&*->><<(^){}[]|\\
    '''
    image = cv2.imread(image_file)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text_extracted=pytesseract.image_to_string(gray_image,lang='deu',config=custom_config)
    text_extracted=pytesseract.image_to_string(gray_image,lang='deu',config=custom_config)
    return text_extracted

In [ ]:
import csv
import os
from PIL import Image
import pytesseract
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Specify the main folder containing subfolders with images
main_folder = '/content/products_leaflets_512/test'

# Create a CSV file
csv_filename = "ocr_result_test.csv"

# Write a header row to the CSV file
header = ["Image File", "PSM6 RGB","PSM6 Gray","PSM12 RGB","PSM 12 Gray","Class Name"]

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)



    # Function to process a single image file
    def process_image(image_file, folder_name):
        image_path = os.path.join(main_folder, folder_name, image_file)
        ocr_result_psm6_RGB = textextraction_psm6_RGB(image_path)
        ocr_result_psm6_Gray = textextraction_psm6_Gray(image_path)
        ocr_result_psm11_RGB = textextraction_psm12_RGB(image_path)
        ocr_result_psm11_Gray = textextraction_psm12_Gray(image_path)
        row = [image_file, ocr_result_psm6_RGB,ocr_result_psm6_Gray, ocr_result_psm11_RGB, ocr_result_psm11_Gray, folder_name]
        csv_writer.writerow(row)

    # Iterate through all subfolders in the main folder
    for folder_name in tqdm(os.listdir(main_folder)):
        folder_path = os.path.join(main_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Get a list of image files in the subfolder
            image_files_to_process = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            # Use ThreadPoolExecutor for parallel processing
            with ThreadPoolExecutor() as executor:
                # Process each image file in parallel
                executor.map(process_image, image_files_to_process, [folder_name]*len(image_files_to_process))

print(f"Results written to {csv_filename}")


In [1]:
import pandas as pd

# Read the first CSV file
csv_file_path1 = '/content/ocr_resulttrain_psm11Gray.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')

# Read the second CSV file
csv_file_path2 = '/content/ocr_resulttrain_psm11RGB.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')

# Merge the two DataFrames on the "Image File" column
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
# Save the merged DataFrame to an Excel file
csv_file_path = 'output_1.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")


Combined data saved to output_1.csv


In [2]:
merged_df.head()

,Image File,PSM11 gray,Class Name_left,PSM11 RGB,Class Name_right
0,8047.jpg,"â â Oldesloer Weizenkorn 32% vol 0,7 ...",150.0,e â â Oldesloer Weizenkorn 32% vol 0...,150.0
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, âS ...",150.0,"PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",150.0
2,8038.jpg,"iiâ Sieeioat I âenkorn = aeslon, \ D...",150.0,"Oldesloer Weizenkorn 2 ol, ori ii erflas...",150.0
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...",150.0,"Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...",150.0
4,8037.jpg,"a â Oldesloer Weizenkorn 32% vol, 0,7Li...",150.0,"Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...",150.0


In [2]:
import pandas as pd

# Read the first CSV file
csv_file_path1 = '/content/ocr_result_train_psm3RGB.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')

# Read the second CSV file
csv_file_path2 = '/content/ocr_result_train_psm3gray.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')

# Merge the two DataFrames on the "Image File" column
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
# Save the merged DataFrame to an Excel file
csv_file_path = 'output_2.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")

Combined data saved to output_2.csv


In [4]:
merged_df.head()

,Image File,PSM3 RGB,Class Name_left,PSM3 Gray,Class Name_right
0,35583.jpg,"geschnliten, I GELESEN HOLTEREN """,672,"geschnliten, ES TAT R:3",672
1,35585.jpg,Anno â1Ã¶HBe ELCH EL = er t == ...,672,"Harry âAnno â188Bx ER ET A Kikg=2,...",672
2,35580.jpg,Harry 1688 Steinofenbrot,672,"Harry 1688 Steinofenbrot 1kg=1,98 Â®",672
3,35555.jpg,Harry 1688 Mehrkornbrot geschnitten je 500 g ...,672,Harry 1688 Mehrkornbrot geschnitten,672
4,35593.jpg,,672,Harry 1688 Mehrkombrot 500g Beutel Ikg=198,672


In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/content/output_1.csv'
df = pd.read_csv(csv_file_path)

# Specify the column to be deleted
column_to_delete = 'Class Name_right'

# Check if the column exists before deleting
if column_to_delete in df.columns:
    # Drop the specified column
    df = df.drop(columns=[column_to_delete])

    # Save the modified DataFrame to a new CSV file
    new_csv_file_path = 'output_1.csv'
    df.to_csv(new_csv_file_path, index=False)

    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")


Column 'Class Name_right' deleted. New data saved to output_1.csv


In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/content/output_2.csv'
df = pd.read_csv(csv_file_path)

# Specify the column to be deleted
column_to_delete = 'Class Name_left'

# Check if the column exists before deleting
if column_to_delete in df.columns:
    # Drop the specified column
    df = df.drop(columns=[column_to_delete])

    # Save the modified DataFrame to a new CSV file
    new_csv_file_path = 'output_2.csv'
    df.to_csv(new_csv_file_path, index=False)

    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")

Column 'Class Name_left' deleted. New data saved to output_2.csv


In [7]:
df.head()

,Image File,PSM3 RGB,PSM3 Gray,Class Name_right
0,35583.jpg,"geschnliten, I GELESEN HOLTEREN ""","geschnliten, ES TAT R:3",672
1,35585.jpg,Anno â1Ã¶HBe ELCH EL = er t == ...,"Harry âAnno â188Bx ER ET A Kikg=2,...",672
2,35580.jpg,Harry 1688 Steinofenbrot,"Harry 1688 Steinofenbrot 1kg=1,98 Â®",672
3,35555.jpg,Harry 1688 Mehrkornbrot geschnitten je 500 g ...,Harry 1688 Mehrkornbrot geschnitten,672
4,35593.jpg,,Harry 1688 Mehrkombrot 500g Beutel Ikg=198,672


In [5]:
import pandas as pd

# Read the first CSV file
csv_file_path1 = '/content/output_1.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')

# Read the second CSV file
csv_file_path2 = '/content/output_2.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')

# Merge the two DataFrames on the "Image File" column
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
# Save the merged DataFrame to an Excel file
csv_file_path = 'ocr_result.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")

Combined data saved to ocr_result.csv


In [9]:
merged_df.head()

,Image File,PSM11 gray,Class Name_left,PSM11 RGB,PSM3 RGB,PSM3 Gray,Class Name_right
0,8047.jpg,Ã¢ÂÂ Ã¢ÂÂ Oldesloer Weizenkorn 32% vol...,150.0,e Ã¢ÂÂ Ã¢ÂÂ Oldesloer Weizenkorn 32% ...,"Oldesloer Weizenkorn 32% vol 0,7 I Flasche...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...",150.0
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, Ã¢ÂÂ...",150.0,"PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",23_11,"za IE S een 32% vol, je 0,7Liter Fla...",150.0
2,8038.jpg,iiÃ¢ÂÂ Sieeioat I Ã¢ÂÂenkorn = aeslon...,150.0,"Oldesloer Weizenkorn 2 ol, ori ii erflas...","Oldesloer Weizenkorn 32% vol, 07",Oldesloer Weizenkorn,150.0
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...",150.0,"Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter rn v...","Oldesioher Weizenkorn 32% vol, 0,7 Liter an ...",150.0
4,8037.jpg,"a Ã¢ÂÂ Oldesloer Weizenkorn 32% vol, 0,...",150.0,"Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...",150.0


In [6]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/content/ocr_result.csv'
df = pd.read_csv(csv_file_path)

# Specify the column to be deleted
column_to_delete = 'Class Name_left'

# Check if the column exists before deleting
if column_to_delete in df.columns:
    # Drop the specified column
    df = df.drop(columns=[column_to_delete])

    # Save the modified DataFrame to a new CSV file
    new_csv_file_path = 'ocr_result.csv'
    df.to_csv(new_csv_file_path, index=False)

    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")

Column 'Class Name_left' deleted. New data saved to ocr_result.csv


In [11]:
df.head()

,Image File,PSM11 gray,PSM11 RGB,PSM3 RGB,PSM3 Gray,Class Name_right
0,8047.jpg,Ã¢ÂÂ Ã¢ÂÂ Oldesloer Weizenkorn 32% vol...,e Ã¢ÂÂ Ã¢ÂÂ Oldesloer Weizenkorn 32% ...,"Oldesloer Weizenkorn 32% vol 0,7 I Flasche...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...",150.0
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, Ã¢ÂÂ...","PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",23_11,"za IE S een 32% vol, je 0,7Liter Fla...",150.0
2,8038.jpg,iiÃ¢ÂÂ Sieeioat I Ã¢ÂÂenkorn = aeslon...,"Oldesloer Weizenkorn 2 ol, ori ii erflas...","Oldesloer Weizenkorn 32% vol, 07",Oldesloer Weizenkorn,150.0
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter rn v...","Oldesioher Weizenkorn 32% vol, 0,7 Liter an ...",150.0
4,8037.jpg,"a Ã¢ÂÂ Oldesloer Weizenkorn 32% vol, 0,...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...",150.0


In [7]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/content/ocr_result.csv'
df = pd.read_csv(csv_file_path)

# Define a function to clean symbols in each cell
def clean_symbols(cell_value):
    # Check if the value is a string
    if isinstance(cell_value, str):
        # Replace or remove unknown symbols based on your requirement
        cleaned_value = cell_value.replace('Ã¢ÂÂ', '').replace('Ã¢ÂÂ', '').replace('Ã¢ÂÂ',' ')
        return cleaned_value
    else:
        # If the value is not a string (e.g., float), return it as is or convert to string
        return cell_value

# Apply the clean_symbols function to each cell in the DataFrame
df_cleaned = df.applymap(clean_symbols)

# Save the cleaned DataFrame to a new CSV file
new_csv_file_path = 'train.csv'
df_cleaned.to_csv(new_csv_file_path, index=False)

print(f"Unknown symbols removed. Cleaned data saved to {new_csv_file_path}")



Unknown symbols removed. Cleaned data saved to train.csv


In [13]:
df_cleaned.head()

,Image File,PSM11 gray,PSM11 RGB,PSM3 RGB,PSM3 Gray,Class Name_right
0,8047.jpg,"Oldesloer Weizenkorn 32% vol 0,7 I Flas...","e Oldesloer Weizenkorn 32% vol 0,7 I ...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...",150.0
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, Ã¢ÂÂ...","PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",23_11,"za IE S een 32% vol, je 0,7Liter Fla...",150.0
2,8038.jpg,iiÃ¢ÂÂ Sieeioat I Ã¢ÂÂenkorn = aeslon...,"Oldesloer Weizenkorn 2 ol, ori ii erflas...","Oldesloer Weizenkorn 32% vol, 07",Oldesloer Weizenkorn,150.0
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter rn v...","Oldesioher Weizenkorn 32% vol, 0,7 Liter an ...",150.0
4,8037.jpg,"a Oldesloer Weizenkorn 32% vol, 0,7Liter...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...",150.0


In [8]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)

# Define a function to combine values from specified columns
def combine_columns(row):
    return f"{row['PSM11 RGB']} {row['PSM11 gray']} {row['PSM3 RGB']} {row['PSM3 Gray']}"

# Apply the combine_columns function to create a new 'combined_column'
df['combined_column'] = df.apply(combine_columns, axis=1)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Columns combined. Modified data saved to {new_csv_file_path}")


Columns combined. Modified data saved to train.csv


In [15]:
df.head()

,Image File,PSM11 gray,PSM11 RGB,PSM3 RGB,PSM3 Gray,Class Name_right,combined_column
0,8047.jpg,"Oldesloer Weizenkorn 32% vol 0,7 I Flas...","e Oldesloer Weizenkorn 32% vol 0,7 I ...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...",150.0,"e Oldesloer Weizenkorn 32% vol 0,7 I ..."
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, Ã¢ÂÂ...","PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",23_11,"za IE S een 32% vol, je 0,7Liter Fla...",150.0,"PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl..."
2,8038.jpg,iiÃ¢ÂÂ Sieeioat I Ã¢ÂÂenkorn = aeslon...,"Oldesloer Weizenkorn 2 ol, ori ii erflas...","Oldesloer Weizenkorn 32% vol, 07",Oldesloer Weizenkorn,150.0,"Oldesloer Weizenkorn 2 ol, ori ii erflas..."
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter rn v...","Oldesioher Weizenkorn 32% vol, 0,7 Liter an ...",150.0,"Oldesloher Weizenkorn 32% vol, 0,7 Liter wa..."
4,8037.jpg,"a Oldesloer Weizenkorn 32% vol, 0,7Liter...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...",150.0,"Oldesloer Weizenkorn 32% vol 0,7LiterFlasch..."


In [9]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/content/ocr_result_test.csv'
df = pd.read_csv(csv_file_path)

# Define a function to combine values from specified columns
def combine_columns(row):
    return f"{row['PSM11 RGB']} {row['PSM11 Gray']} {row['PSM3 RGB']} {row['PSM3 Gray']}"

# Apply the combine_columns function to create a new 'combined_column'
df['combined_column'] = df.apply(combine_columns, axis=1)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Columns combined. Modified data saved to {new_csv_file_path}")


Columns combined. Modified data saved to test.csv


In [10]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)

# Specify the column you want to clean
column_to_clean = 'combined_column'

# Define a function to keep only German characters and numbers
def keep_german_and_numbers(cell_value):
    # Keep only characters in the German language or numbers
    cleaned_value = re.sub(r'[^a-zA-ZäöüÄÖ%=Üß0-9]', '  ', str(cell_value))
    return cleaned_value

# Apply the keep_german_and_numbers function to the specified column
df[column_to_clean] = df[column_to_clean].apply(keep_german_and_numbers)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Column '{column_to_clean}' cleaned. Modified data saved to {new_csv_file_path}")


Column 'combined_column' cleaned. Modified data saved to train.csv


In [13]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
csv_file_path = 'test.csv'
df = pd.read_csv(csv_file_path)

# Specify the column you want to clean
column_to_clean = 'combined_column'

# Define a function to keep only German characters and numbers
def keep_german_and_numbers(cell_value):
    # Keep only characters in the German language or numbers
    cleaned_value = re.sub(r'[^a-zA-ZäöüÄÖ%=Üß0-9]', '  ', str(cell_value))
    return cleaned_value

# Apply the keep_german_and_numbers function to the specified column
df[column_to_clean] = df[column_to_clean].apply(keep_german_and_numbers)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Column '{column_to_clean}' cleaned. Modified data saved to {new_csv_file_path}")


Column 'combined_column' cleaned. Modified data saved to test.csv


In [14]:
df.head()

,Image File,PSM3 RGB,PSM3 Gray,PSM11 RGB,PSM11 Gray,Class Name,combined_column
0,8045.jpg,"Oldesloer\nWeizenkorn\n32 %Vol,,\n\n \n ...","Oldesloer\nWeizenkorn\n32 %Vol,,\n11=7,13\n\n ...","Oldesloer\n\nWeizenkorn\n\n32 %Vol,,\n\n11=7,1...","Oldesloer\n\nWeizenkorn\n\n32 %Vol,,\n\n11=7,1...",150,Oldesloer Weizenkorn 32 %Vol 11=...
1,8054.jpg,"‚oldesloer\n\nWeizenkorn\n32% vol,\n0,7LiterFl...","\n\nOldesloer\nWeizenkorn\n32% vol,\n0,7Liter...","‚oldesloer\n\nWeizenkorn\n\n32% vol,\n\n9,7Lit...","oldesloer\n\nWeizenkorn\n\n32% vol,\n\n0,TLite...",150,oldesloer Weizenkorn 32% vol 9 ...
2,8075.jpg,Aus sonnen\ngereiftem\nWeizen\nmeisterhaft\nge...,"Oldesloer\n\na Weizenkorn\nAus sonnen 32,0% vo...",Oldesloer\n\nWeizenkorn\n\nAus sonnen\n\nA\n\n...,Oldesloer\n\nu\n\nWeizenkorn\n\nAus sonnen\n\n...,150,Oldesloer Weizenkorn Aus sonnen A ...
3,8044.jpg,= AN\n\nMWeizenkoen\n\n \n,\n\nAKTIONSPREIS\n\nOldesloer\nWeizenkom\n3...,"we\n\n80510,\n\nS\n\nWeizen\n\nhoen\n\n0\n\n—\...","80510,\n\nWeizen\n\nl—\n\nei\n\nAKTIONSPREIS\n...",150,we 80510 S Weizen hoen 0 ...
4,8039.jpg,"Oldesloer\nWeizenkorn\n\n32% Vol,\n\n0,71 Lite...","Oldesloer\nWeizenkorn =: =?\n32%Vol, nn el\nO,...","Weizenkorn\n\nOldesloer\n\n32% Vol,\n\n0,71 Li...","Oldesloer\n\nWeizenkorn\n\n32% Vol,\n\nii\n\n0...",150,Weizenkorn Oldesloer 32% Vol 0 71...


In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [16]:
# Load the Excel file
file_path = 'train.csv'
df = pd.read_csv(file_path)

# Assuming the column names are 'Column1' and 'Column2', replace them with your actual column names
column_name = 'combined_column'

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

# Clean and tokenize the text in the specified column
df[column_name] = df[column_name].apply(clean_and_tokenize)

# Save the cleaned and tokenized DataFrame to a new Excel file
output_file_path = 'train.csv'
df.to_csv(output_file_path, index=False)

In [17]:
# Load the Excel file
file_path = 'test.csv'
df = pd.read_csv(file_path)

# Assuming the column names are 'Column1' and 'Column2', replace them with your actual column names
column_name = 'combined_column'

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

# Clean and tokenize the text in the specified column
df[column_name] = df[column_name].apply(clean_and_tokenize)

# Save the cleaned and tokenized DataFrame to a new Excel file
output_file_path = 'test.csv'
df.to_csv(output_file_path, index=False)

In [26]:
import pandas as pd

# Load the CSV file
csv_file_path = 'train.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Function to remove duplicate words in a string
def remove_duplicates(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))
    return ' '.join(unique_words)

# Apply the function to the 'combined_column'
df['combined_column'] = df['combined_column'].apply(remove_duplicates)

# Save the updated DataFrame back to the CSV file
df.to_csv('train.csv', index=False)

print("Duplicate words within each row of 'combined_column' have been removed.")


Duplicate words within each row of 'combined_column' have been removed.


In [27]:
df.head()

,Image File,PSM11 gray,PSM11 RGB,PSM3 RGB,PSM3 Gray,Class Name_right,combined_column,common_words
0,8047.jpg,"Oldesloer Weizenkorn 32% vol 0,7 I Flas...","e Oldesloer Weizenkorn 32% vol 0,7 I ...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche...",150.0,"['e', 'oldesloer', 'weizenkorn', '32', 'vol', ...","Oldesloer Weizenkorn 32% vol 0,7 I Flasche 8% ..."
1,8061.jpg,"Fu Ba OLDESLOER Weizenkorn 32% vol, Ã¢ÂÂ...","PN Weizenkorn 32% vol, Be je 0,7Liter"" Fl...",23_11,"za IE S een 32% vol, je 0,7Liter Fla...",150.0,"['pn', 'weizenkorn', '32', 'vol', 'be', 'je', ...","Weizenkorn 32% vol, je 0,7Liter Flasche 1Liter..."
2,8038.jpg,iiÃ¢ÂÂ Sieeioat I Ã¢ÂÂenkorn = aeslon...,"Oldesloer Weizenkorn 2 ol, ori ii erflas...","Oldesloer Weizenkorn 32% vol, 07",Oldesloer Weizenkorn,150.0,"['oldesloer', 'weizenkorn', '2', 'ol', 'ori', ...",\ Oldesloer Weizenkorn
3,8057.jpg,"Oldesloher Weizenkorn 32% vol, 0,7 Liter aa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter wa...","Oldesloher Weizenkorn 32% vol, 0,7 Liter rn v...","Oldesioher Weizenkorn 32% vol, 0,7 Liter an ...",150.0,"['oldesloher', 'weizenkorn', '32', 'vol', '0',...","Oldesloher Weizenkorn 32% vol, 0,7 Liter un = ..."
4,8037.jpg,"a Oldesloer Weizenkorn 32% vol, 0,7Liter...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasch...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...","Oldesloer Weizenkorn 32% vol 0,7LiterFlasche ...",150.0,"['oldesloer', 'weizenkorn', '32', 'vol', '0', ...","Oldesloer Weizenkorn 32% 0,7LiterFlasche 1 == ..."


In [18]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)

# List of columns to be deleted
columns_to_delete = ['PSM11 gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray']

# Drop the specified columns
df.drop(columns=columns_to_delete, inplace=True)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")


Columns ['PSM11 gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray'] deleted. Modified data saved to train.csv


In [19]:
df.head()

,Image File,Class Name_right,combined_column
0,8047.jpg,150.0,"['e', 'oldesloer', 'weizenkorn', '32', 'vol', ..."
1,8061.jpg,150.0,"['pn', 'weizenkorn', '32', 'vol', 'be', 'je', ..."
2,8038.jpg,150.0,"['oldesloer', 'weizenkorn', '2', 'ol', 'ori', ..."
3,8057.jpg,150.0,"['oldesloher', 'weizenkorn', '32', 'vol', '0',..."
4,8037.jpg,150.0,"['oldesloer', 'weizenkorn', '32', 'vol', '0', ..."


In [20]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'test.csv'
df = pd.read_csv(csv_file_path)

# List of columns to be deleted
columns_to_delete = ['PSM11 Gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray']

# Drop the specified columns
df.drop(columns=columns_to_delete, inplace=True)

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")

Columns ['PSM11 Gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray'] deleted. Modified data saved to test.csv


In [31]:
df.head(3300)

,Image File,Class Name,combined_column,common_words
0,8045.jpg,150,"['oldesloer', 'weizenkorn', '32', 'vol', '1171...","['oldesloer', 'weizenkorn', '32', 'vol', 'alt'..."
1,8054.jpg,150,"['‚oldesloer', 'weizenkorn', '32', 'vol', '97l...","['oldesloer', 'weizenkorn', '32', 'vol', '0', ..."
2,8075.jpg,150,"['oldesloer', 'weizenkorn', 'sonnen', 'a', '32...","['sonnen', 'gereiftem', 'weizen', 'meisterhaft..."
3,8044.jpg,150,"['we', '80510', 's', 'weizen', 'hoen', '0', '—...","['aktionspreis', 'oldesloer', 'weizenkom', '32..."
4,8039.jpg,150,"['weizenkorn', 'oldesloer', '32', 'vol', '071'...","['oldesloer', 'weizenkorn', '32', 'vol', '0', ..."
...,...,...,...,...
3295,31939.jpg,605,"['7', 'n', 'ar', 'sn', 'ar', '4', 'n', 'dan', ...","['zimmermann', 'weißwurstkult', '10', 'münchne..."
3296,31956.jpg,605,"['art', 'kantlharrt', 'un', 'ir', '2', 'sl', '...","['zimmermann', 'weißwurstkult', 'münchner', 'w..."
3297,31957.jpg,605,"['srtterntlatlanttrroftunnt', 'y', '—', 'a', '...","['zimmermann', 'weißwurstkult', 'münchner', 'w..."
3298,31987.jpg,605,"['zimmermann', 'weißwurst', 'kult', '10', 'mün...","['zimmermann', 'weißwurst', 'kult', '10', 'mün..."


In [21]:
import pandas as pd

# Read the csv file into a DataFrame
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)

# Shuffle the rows of the DataFrame
shuffled_df = df.sample(frac=1).reset_index(drop=True)

# Save the shuffled DataFrame back to the  file
shuffled_csv_file_path = 'train.csv'  # Replace with the desired output path
shuffled_df.to_csv(shuffled_csv_file_path, index=False)

print(f"Rows shuffled and saved to: {shuffled_csv_file_path}")

Rows shuffled and saved to: train.csv


In [22]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)

# Specify the column with NaN values
column_with_nan = 'Class Name_right'

# Drop rows where the specified column has NaN values
df = df.dropna(subset=[column_with_nan])

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Rows with NaN values in column '{column_with_nan}' deleted. Modified data saved to {new_csv_file_path}")


Rows with NaN values in column 'Class Name_right' deleted. Modified data saved to train.csv


In [23]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'test.csv'
df = pd.read_csv(csv_file_path)

# Specify the column with NaN values
column_with_nan = 'Class Name'

# Drop rows where the specified column has NaN values
df = df.dropna(subset=[column_with_nan])

# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)

print(f"Rows with NaN values in column '{column_with_nan}' deleted. Modified data saved to {new_csv_file_path}")

Rows with NaN values in column 'Class Name' deleted. Modified data saved to test.csv


In [24]:
import tensorflow
import pandas as pd

df=pd.read_csv('train.csv')
x_train_data=df['combined_column']
label=df['Class Name_right'].astype(int)
df_test =pd.read_csv('test.csv')
x_validation_data=df_test['combined_column']
label_validation=df_test['Class Name']


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer


x_train_data.fillna('', inplace=True)

# Create and fit the TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train_data)

# Transform the data
x_train_vec = tfidf_vect.transform(x_train_data)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming x_train_data is your text data
# Replace NaN values with an empty string
x_validation_data.fillna('', inplace=True)



In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(x_train_data)
x_train_vec = tfidf_vect.transform(x_train_data)
x_validation_vec = tfidf_vect.transform(x_validation_data)

In [28]:
tfidf_vect.vocabulary_

{'il': 47207,
 'breitsamer': 23384,
 'honig': 44871,
 'hl': 44491,
 'ne': 63523,
 'fr': 38600,
 'hlingssummen': 44566,
 'at': 18574,
 'je': 50133,
 '500': 10143,
 'gglas': 41148,
 'kg': 53075,
 '98': 13233,
 'ar': 17738,
 'rel': 72614,
 'umien': 86226,
 'ei': 30880,
 'glas': 41433,
 'ul': 85994,
 'gg': 41136,
 'haartrimmer': 42750,
 'hc5050': 43486,
 'akkulaufzeit': 15513,
 '50': 10142,
 'min': 61392,
 '17': 4087,
 'ngeneinstellungen': 64252,
 'spezielle': 79497,
 'kammaufs': 51998,
 'tze': 85134,
 'trimmer': 84451,
 '324': 8285,
 'mm': 61878,
 're': 72207,
 'kamm': 51997,
 '1435': 3605,
 'abwaschbar': 14254,
 'ya': 91964,
 'praun': 70300,
 'angebotspreis': 16924,
 'braun': 23320,
 'br': 23226,
 'un': 86314,
 'ill': 47341,
 'coraya': 25869,
 'et': 35348,
 'fish': 37982,
 'dip': 27956,
 'cocktail': 25726,
 'fischerei': 37964,
 '100': 962,
 '93': 13094,
 'old': 66569,
 'sog': 78900,
 'ri': 73118,
 'el': 31984,
 'zertifizierte': 93247,
 'frrtehtr': 39015,
 'aue': 18898,
 'dr': 28513,
 'fi

In [50]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
label_ = label[:]
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(x_train_vec, label_)

SVC(kernel='linear')

In [51]:
y_pred = svm_classifier.predict(x_validation_vec)
accuracy = accuracy_score(label_validation, y_pred)
print(accuracy)

0.7997596153846154


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have your text data in X and corresponding labels in y
# X should be a list of strings, and y should be a list of labels

# Split the data into training and validation sets


# Create a pipeline with TF-IDF vectorizer and SGDClassifier
model = make_pipeline(
    TfidfVectorizer(),
    SGDClassifier(loss='modified_huber', alpha=0.0001, max_iter=100, random_state=42, learning_rate='adaptive',eta0=0.01)
)

# Train the model on the training data
model.fit(x_train_data, label[:])

# Make predictions on the validation set
y_pred = model.predict(x_validation_data)

# Calculate accuracy on the validation set
accuracy = accuracy_score(label_validation, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy}")


Accuracy: 0.815625


In [29]:
from sklearn.metrics import accuracy_score

In [35]:
from sklearn.naive_bayes import MultinomialNB

nb_classifier = MultinomialNB()
nb_classifier.fit(x_train_vec, label[:])

# Prediction
y_pred = nb_classifier.predict(x_validation_vec)

# Accuracy
accuracy = accuracy_score(label_validation, y_pred)
print("Multinomial Naive Bayes Accuracy:", accuracy)


Multinomial Naive Bayes Accuracy: 0.8305288461538461


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Instantiate the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Define the hyperparameters to tune
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Smoothing parameter
    'fit_prior': [True, False]  # Whether to learn class prior probabilities from the data
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train_vec, label[:])

# Get the best parameters and best accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)



Best Parameters: {'alpha': 0.1, 'fit_prior': True}
Best Accuracy: 0.9383112980769232


In [34]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Instantiate the Multinomial Naive Bayes classifier with the best parameters
best_nb_classifier = MultinomialNB(alpha=0.1, fit_prior=True)

# Train the classifier on the entire training dataset
best_nb_classifier.fit(x_train_vec, label[:])

# Perform prediction on the test dataset
y_pred_test = best_nb_classifier.predict(x_validation_vec)

# Calculate accuracy on the test dataset
accuracy_test = accuracy_score(label_validation, y_pred_test)
print("Accuracy on Test Set:", accuracy_test)


Accuracy on Test Set: 0.8284855769230769


In [37]:
from joblib import dump
model_filename = 'multinomial_naive_bayes_model.joblib'
dump(nb_classifier, model_filename)
print("Model saved as:", model_filename)

Model saved as: multinomial_naive_bayes_model.joblib


In [ ]:


import joblib
joblib.dump(model, 'sgd_model_new.joblib')

['sgd_model_new.joblib']

In [ ]:
joblib.dump(model, 'sgd_model_new.joblib')

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
source='/content/sgd_model_new.joblib'
dest_path_drive = '/content/gdrive/My Drive/'

In [ ]:
import shutil
shutil.copy(source, dest_path_drive)

'/content/gdrive/My Drive/sgd_model_new.joblib'

In [ ]:
source_path_tfidf = 'tfidf_vectorizer_1.joblib'

dest_path_drive = '/content/gdrive/My Drive/'

In [ ]:
import shutil
shutil.copy(source_path_tfidf_1, dest_path_drive)


NameError: name 'source_path_tfidf_1' is not defined